In [1]:
# Import necessary libraries for data manipulation and analysis

import re
import joblib
import numpy as np
import pandas as pd
import xgboost as xgb
import category_encoders as ce
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import VotingRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# Read the CSV files containing player data

players_22 = pd.read_csv('players_22.csv')
male_players_legacy = pd.read_csv('male_players (legacy).csv')

C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\1340866027.py:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('players_22.csv')
C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\1340866027.py:4: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  male_players_legacy = pd.read_csv('male_players (legacy).csv')


In [3]:
# A Function For Cleaning Datasets For Training
def clean_data(df,condition):

    # Drop columns containing 'url' or 'dob' in their names
    columns_to_drop = df.filter(regex='url|dob').columns
    df.drop(columns=columns_to_drop, axis=1, inplace=True)

    # Create lists to hold columns with less than 30% missing values and columns with more than 30% missing values
    L = []
    L_less = []
    for i in df.columns:
        if df[i].isnull().sum()<(0.3 * df.shape[0]):
            L.append(i)
        else:
            L_less.append(i)

    df = df[L]

    # Impute missing values: use the median for numeric columns and the most frequent value for object columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    num_imputer = SimpleImputer(strategy='median')
    df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])


    obj_imputer = SimpleImputer(strategy='most_frequent')
    df[df.select_dtypes(include='object').columns] = obj_imputer.fit_transform\
        (df[df.select_dtypes
    (include='object').columns])

    if condition == "Yes":
        # Drop specified columns
        columns_to_drop = ['player_id','fifa_version','short_name', 'long_name','player_positions','club_team_id',
                           'club_name','club_joined_date','club_contract_valid_until_year','league_name',
                           'league_level','club_position','club_jersey_number','fifa_update','fifa_update_date'
                           ,'nationality_id','nationality_name','real_face']
        df.drop(columns=columns_to_drop, axis=1, inplace=True)

    # Convert specific patterns in the columns from string to numerical values
    for col in df.columns:
        # Convert column to string for checking patterns
        col_str = df[col].astype(str)

        # Check if the column contains any data in the form of "number+number" or "number-number"
        if col_str.str.contains(r'^\d+[\+\-]\d+$').any():
            # Apply eval to each element in the column if it matches the pattern
            df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) and re.match
            (r'^\d+[\+\-]\d+$', x) else x)

            # Convert the column to int or float as needed
            if df[col].apply(lambda x: isinstance(x, (int, float))).all():
                df[col] = df[col].astype(float)

    # Encode object columns using BinaryEncoder
    for column in df.columns[:69]:
        if df[column].dtype == 'object':
            encoder = ce.BinaryEncoder(cols=[column])
            col_encoded = encoder.fit_transform(df[column])
            df.drop(column, axis=1, inplace=True)
            df = pd.concat([df, col_encoded], axis=1).reset_index(drop=True)

    return df

In [4]:
# Clean the first dataset

male_players_legacy = clean_data(male_players_legacy, "Yes")

C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\3199309306.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])
C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\3199309306.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.select_dtypes(include='object').columns] = obj_imputer.fit_transform\
C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\3199309306.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [5]:
# Clean the second dataset

players_22 = clean_data(players_22, "No")

C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\3199309306.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])
C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\3199309306.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.select_dtypes(include='object').columns] = obj_imputer.fit_transform\
C:\Users\mcnob\AppData\Local\Temp\ipykernel_15600\3199309306.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [6]:
# Display the DataFrame

male_players_legacy

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,weak_foot,skill_moves,...,cm_6,cm_7,rcm_0,rcm_1,rcm_2,rcm_3,rcm_4,rcm_5,rcm_6,rcm_7
0,93.0,95.0,100500000.0,550000.0,27.0,169.0,67.0,53.0,3.0,4.0,...,0,1,0,0,0,0,0,0,0,1
1,92.0,92.0,79000000.0,375000.0,29.0,185.0,80.0,53.0,4.0,5.0,...,1,0,0,0,0,0,0,0,1,0
2,90.0,90.0,54500000.0,275000.0,30.0,180.0,80.0,19.0,2.0,4.0,...,1,1,0,0,0,0,0,0,1,1
3,90.0,90.0,52500000.0,275000.0,32.0,195.0,95.0,16.0,4.0,4.0,...,0,0,0,0,0,0,0,1,0,0
4,90.0,90.0,63500000.0,300000.0,28.0,193.0,92.0,19.0,4.0,1.0,...,0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,61.0,110000.0,700.0,18.0,180.0,73.0,2012.0,3.0,2.0,...,1,0,0,1,1,1,1,0,1,0
161579,46.0,58.0,110000.0,750.0,19.0,188.0,83.0,2012.0,3.0,2.0,...,0,1,0,1,1,0,1,0,0,1
161580,46.0,58.0,110000.0,500.0,19.0,181.0,73.0,65.0,2.0,2.0,...,0,1,1,0,0,0,0,0,0,1
161581,46.0,70.0,150000.0,500.0,17.0,175.0,68.0,65.0,3.0,2.0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
# Calculate correlations of all features with the 'overall' column and keep the top 12 features

correlations = male_players_legacy.corr()['overall']
sorted_correlations = correlations.sort_values(ascending=False)

male_players_legacy_relevant_columns = sorted_correlations[:16]

male_players_legacy_relevant_columns.index

Index(['overall', 'movement_reactions', 'potential', 'passing', 'wage_eur',
       'mentality_composure', 'value_eur', 'dribbling',
       'attacking_short_passing', 'mentality_vision',
       'international_reputation', 'skill_long_passing', 'ldm', 'rdm', 'cdm',
       'power_shot_power'],
      dtype='object')

In [8]:
# Keep only the top correlated features

male_players_legacy = male_players_legacy[male_players_legacy_relevant_columns.index]

In [9]:
# Display the updated DataFrame

male_players_legacy

,overall,movement_reactions,potential,passing,wage_eur,mentality_composure,value_eur,dribbling,attacking_short_passing,mentality_vision,international_reputation,skill_long_passing,ldm,rdm,cdm,power_shot_power
0,93.0,94.0,95.0,86.0,550000.0,59.0,100500000.0,96.0,89.0,90.0,5.0,76.0,65,65,65,80.0
1,92.0,90.0,92.0,81.0,375000.0,59.0,79000000.0,91.0,82.0,81.0,5.0,72.0,66,66,66,94.0
2,90.0,89.0,90.0,83.0,275000.0,59.0,54500000.0,92.0,86.0,84.0,5.0,76.0,67,67,67,86.0
3,90.0,85.0,90.0,81.0,275000.0,59.0,52500000.0,86.0,84.0,83.0,5.0,76.0,68,68,68,93.0
4,90.0,89.0,90.0,58.0,300000.0,59.0,63500000.0,63.0,42.0,20.0,5.0,41.0,43,43,43,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,39.0,61.0,40.0,700.0,40.0,110000.0,47.0,38.0,42.0,1.0,33.0,34,34,34,64.0
161579,46.0,42.0,58.0,29.0,750.0,35.0,110000.0,34.0,30.0,31.0,1.0,25.0,43,43,43,33.0
161580,46.0,50.0,58.0,43.0,500.0,35.0,110000.0,46.0,51.0,40.0,1.0,43.0,47,47,47,51.0
161581,46.0,45.0,70.0,36.0,500.0,43.0,150000.0,46.0,40.0,44.0,1.0,26.0,32,32,32,42.0


In [10]:
# Display the first few rows of the players_22 DataFrame

players_22

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,work_rate_0,work_rate_1,work_rate_2,work_rate_3,body_type_0,body_type_1,body_type_2,body_type_3,real_face_0,real_face_1
0,158023.0,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,1.0,...,0,0,0,1,0,0,0,1,0,1
1,188545.0,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,1.0,...,0,0,1,0,0,0,0,1,0,1
2,20801.0,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,1.0,...,0,0,1,1,0,0,0,1,0,1
3,190871.0,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,1.0,...,0,0,1,0,0,0,0,1,0,1
4,192985.0,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,1.0,...,0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962.0,47.0,52.0,70000.0,1000.0,22.0,180.0,64.0,112541.0,1.0,...,0,1,0,1,0,0,1,1,1,0
19235,262040.0,47.0,59.0,110000.0,500.0,19.0,175.0,70.0,445.0,1.0,...,0,1,0,1,0,0,1,1,1,0
19236,262760.0,47.0,55.0,100000.0,500.0,21.0,178.0,72.0,111131.0,1.0,...,0,1,0,1,0,0,1,1,1,0
19237,262820.0,47.0,60.0,110000.0,500.0,19.0,173.0,66.0,111131.0,1.0,...,0,1,0,1,0,0,1,1,1,0


In [11]:
# Define target and features for model training

y = male_players_legacy['overall']
X = male_players_legacy.drop('overall', axis=1)

In [12]:
# Split data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Scale the data

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled data back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Save the scaler for future use
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [14]:
# Display the first few rows of the training set

X_train.head()

,movement_reactions,potential,passing,wage_eur,mentality_composure,value_eur,dribbling,attacking_short_passing,mentality_vision,international_reputation,skill_long_passing,ldm,rdm,cdm,power_shot_power
69722,72.0,77.0,51.0,40000.0,71.0,9000000.0,68.0,63.0,58.0,2.0,37.0,53,53,53,83.0
145131,76.0,77.0,66.0,45000.0,72.0,5500000.0,75.0,70.0,67.0,1.0,58.0,57,57,57,77.0
97854,47.0,71.0,58.0,5000.0,44.0,475000.0,63.0,25.0,35.0,1.0,26.0,23,23,23,49.0
77978,64.0,66.0,41.0,2000.0,68.0,475000.0,45.0,54.0,34.0,1.0,51.0,61,61,61,32.0
105151,80.0,82.0,74.0,60000.0,76.0,27000000.0,79.0,82.0,73.0,2.0,71.0,81,81,81,71.0


In [15]:
y_train.head()

69722     76.0
145131    74.0
97854     64.0
77978     66.0
105151    82.0
Name: overall, dtype: float64

In [16]:
# Multi Linear Regression

# Initialize the Linear Regression model
l = LinearRegression()

In [17]:
# Fit the Linear Regression model on the training data

l.fit(X_train,y_train)

LinearRegression()

In [18]:
# Predict the target values on the test data using the Linear Regression model

y_pred = l.predict(X_test)

In [19]:
# Get the feature names from the training data

X_train.columns

Index(['movement_reactions', 'potential', 'passing', 'wage_eur',
       'mentality_composure', 'value_eur', 'dribbling',
       'attacking_short_passing', 'mentality_vision',
       'international_reputation', 'skill_long_passing', 'ldm', 'rdm', 'cdm',
       'power_shot_power'],
      dtype='object')

In [20]:
# Get the intercept and coefficients of the trained Linear Regression model

# Get the intercept and coefficients of the trained Linear Regression model
intercept = l.intercept_
coefficients = l.coef_

# Print the intercept and coefficients of the Linear Regression model
print(f"Intercept: {intercept}")
print(f"Coefficients: {coefficients}")

Intercept: 7.634738574625672
Coefficients: [ 3.44609873e-01  3.13662027e-01  1.86695751e-01  3.02202744e-05
  7.91780381e-02 -6.84998056e-09 -2.62118117e-03 -4.25028919e-02
 -5.83415858e-02  4.38907694e-01 -6.42944905e-02  3.12044169e-02
  3.12044169e-02  3.12044169e-02  4.58559116e-02]


In [21]:
# Evaluate the Linear Regression model using various metrics

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.1684700140114983,
Mean Squared Error = 7.854723831250511,
Root Mean Squared Error = 2.802628022276683,
R2 Score = 0.8121773376485313



In [22]:
# Polynomial Regression

# Create polynomial regression pipelines with degree 1 and 2
poly_pipeline1 = make_pipeline(PolynomialFeatures(degree=1), LinearRegression())
poly_pipeline2 = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())

# Transform the training and test data to include polynomial features (degree 1)
poly1 = PolynomialFeatures(degree=1)
X_poly_train = poly1.fit_transform(X_train)
X_poly_test = poly1.fit_transform(X_test)

# Initialize and fit a Linear Regression model on the polynomial transformed data
model = LinearRegression()
model.fit(X_poly_train, y_train)

# Predict the target values on the polynomial transformed test data
y_pred = model.predict(X_poly_test)

In [23]:
# Evaluate the Polynomial Regression model (degree 1) using various metrics

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred, y_test)},
Mean Squared Error = {mean_squared_error(y_pred, y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, y_test))},
R2 Score = {r2_score(y_pred, y_test)}
""")


Mean Absolute Error = 2.1684700140170174,
Mean Squared Error = 7.854723831239558,
Root Mean Squared Error = 2.8026280222747286,
R2 Score = 0.8121773376601387



In [24]:
# Transform the training and test data to include polynomial features (degree 2)

poly2 = PolynomialFeatures(degree=2)
X_poly_train = poly2.fit_transform(X_train)
X_poly_test = poly2.fit_transform(X_test)

# Initialize and fit a Linear Regression model on the polynomial transformed data
model = LinearRegression()
model.fit(X_poly_train, y_train)

# Predict the target values on the polynomial transformed test data
y_pred = model.predict(X_poly_test)

# Evaluate the Polynomial Regression model (degree 2) using various metrics
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred, y_test)},
Mean Squared Error = {mean_squared_error(y_pred, y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, y_test))},
R2 Score = {r2_score(y_pred, y_test)}
""")


Mean Absolute Error = 1.494176354157657,
Mean Squared Error = 3.944942825185737,
Root Mean Squared Error = 1.9861880135540384,
R2 Score = 0.9147453194736997



In [25]:
# Transform the training and test data to include polynomial features (degree 3)

poly3 = PolynomialFeatures(degree=3)
X_poly_train = poly3.fit_transform(X_train)
X_poly_test = poly3.fit_transform(X_test)

# Initialize and fit a Linear Regression model on the polynomial transformed data
model = LinearRegression()
model.fit(X_poly_train, y_train)

# Predict the target values on the polynomial transformed test data
y_pred = model.predict(X_poly_test)

# Evaluate the Polynomial Regression model (degree 3) using various metrics
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred, y_test)},
Mean Squared Error = {mean_squared_error(y_pred, y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, y_test))},
R2 Score = {r2_score(y_pred, y_test)}
""")


Mean Absolute Error = 3.3222822794181712,
Mean Squared Error = 24.42133423672465,
Root Mean Squared Error = 4.9417946372471455,
R2 Score = 0.44601529132064466



In [26]:
# Regularisation Models

# Ridge Regression

# Initialize Ridge regression mode
model1 = Ridge()

# Fit the model on the training data
model1.fit(X_train, y_train)

# Predict the target values on the test data
y_pred = model1.predict(X_test)

C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.96323e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [27]:
# Evaluate the Ridge Regression model using various metrics

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.1684698562324014,
Mean Squared Error = 7.854723812890498,
Root Mean Squared Error = 2.8026280190011836,
R2 Score = 0.812177303410628



In [28]:
# Lasso Regression

# Initialize Lasso regression model
model2 = Lasso()

# Fit the model on the training data
model2.fit(X_train, y_train)

# Predict the target values on the test data
y_pred = model2.predict(X_test)

# Evaluate the Lasso Regression model using various metrics
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.196842115123514,
Mean Squared Error = 8.152581709130308,
Root Mean Squared Error = 2.855272615553602,
R2 Score = 0.7944944954891554



In [29]:
# ElasticNet Regression

# Initialize ElasticNet regression model
model3 = ElasticNet()

# Fit the model on the training data
model3.fit(X_train, y_train)

# Predict the target values on the test data
y_pred = model3.predict(X_test)

# Evaluate the ElasticNet Regression model using various metrics
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.174394552345393,
Mean Squared Error = 7.9573649879627455,
Root Mean Squared Error = 2.820880179653639,
R2 Score = 0.8031806131099133



C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.475e+03, tolerance: 6.408e+02
  model = cd_fast.enet_coordinate_descent(


In [30]:
# Decision Tree Regression

# Initialize Decision Tree Regressor with max depth of 2000
dtree = DecisionTreeRegressor(max_depth=2000)

In [31]:
# Fit the model on the training data

dtree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2000)

In [32]:
# Predict the target values on the test data
y_pred = dtree.predict(X_test)

# Evaluate the Decision Tree Regression model using various metrics
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 1.003032459696135,
Mean Squared Error = 3.011062289197636,
Root Mean Squared Error = 1.7352412769403671,
R2 Score = 0.9394726904748758



In [33]:
# Stacking Regressor

# Define base models for stacking
estimators = [
    ('lr', l),
    ('poly1', poly_pipeline1),
    ('poly2', poly_pipeline2),
    ('rm1', model1),
    ('rm2', model2),
    ('rm3', model3),
    ('dt', dtree)
]

# Initialize Stacking Regressor with Decision Tree as final estimator
stacking_model = StackingRegressor(estimators=estimators, final_estimator=DecisionTreeRegressor
(max_depth=10000))

# Fit the stacking model on the training data
stacking_model.fit(X_train, y_train)

# Predict the target values on the test data using the stacking model
y_pred = stacking_model.predict(X_test)

# Evaluate the Stacking Regressor model using various metrics
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")

C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.96323e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.475e+03, tolerance: 6.408e+02
  model = cd_fast.enet_coordinate_descent(
C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.23046e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarni


Mean Absolute Error = 1.4133118791967076,
Mean Squared Error = 4.323544883497849,
Root Mean Squared Error = 2.079313560648766,
R2 Score = 0.9130635785568475



In [34]:
# Cross-validation scores for the stacking model

cv_scores = cross_val_score(stacking_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)


# Print the mean and standard deviation of cross-validation scores
print(f"Mean CV Score (Negative MSE): {mean_cv_score}")
print(f"Standard Deviation of CV Scores: {std_cv_score}")

Mean CV Score (Negative MSE): -4.344142832102806
Standard Deviation of CV Scores: 0.06503302514400534


In [35]:
# Grid search for hyperparameter tuning of the final estimator in the stacking model

param_grid = {
    'final_estimator__max_depth': [5, 10, 15, 20],
    'final_estimator__min_samples_split': [2, 5, 10]
}


grid_search = GridSearchCV(stacking_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search on the training data
grid_search.fit(X_train_scaled, y_train)

# Best estimator from the grid search
test_model_1 = grid_search.best_estimator_

# Predict the target values on the test data using the best estimator
y_pred_best = test_model_1.predict(X_test_scaled)

# Evaluate the best estimator model using various metrics
print(f"Test Model 1 Mean Absolute Error: {mean_absolute_error(y_test, y_pred_best)}")
print(f"Test Model 1 Mean Squared Error: {mean_squared_error(y_test, y_pred_best)}")
print(f"Test Model 1 Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_best))}")
print(f"Test Model 1 R2 Score: {r2_score(y_test, y_pred_best)}")


Test Model 1 Mean Absolute Error: 1.0226349079054278
Test Model 1 Mean Squared Error: 2.212417907217195
Test Model 1 Root Mean Squared Error: 1.4874198826213112
Test Model 1 R2 Score: 0.9553619850713773


In [36]:
# Refined grid search for hyperparameter tuning of the best estimator

refined_param_grid = {
    'final_estimator__max_depth': [12, 15, 18],
    'final_estimator__min_samples_split': [5, 7, 10]
}

refined_grid_search = GridSearchCV(test_model_1, refined_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the refined grid search on the training data
refined_grid_search.fit(X_train_scaled, y_train)

# Best estimator from the refined grid search
test_model_2 = refined_grid_search.best_estimator_

# Fit the model on the training data
test_model_2.fit(X_train_scaled, y_train)

# Predict the target values on the test data using the model
y_pred_final = test_model_2.predict(X_test_scaled)

# Evaluate the model using various metrics
print(f"Test Model 2 Mean Absolute Error: {mean_absolute_error(y_test, y_pred_final)}")
print(f"Test Model 2 Mean Squared Error: {mean_squared_error(y_test, y_pred_final)}")
print(f"Test Model 2 Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_final))}")
print(f"Test Model 2 R2 Score: {r2_score(y_test, y_pred_final)}")

Test Model 2 Mean Absolute Error: 1.0515895474447812
Test Model 2 Mean Squared Error: 2.3489147126688312
Test Model 2 Root Mean Squared Error: 1.532616949100078
Test Model 2 R2 Score: 0.952608008790683


In [37]:
# Define refined parameter grid for hyperparameter tuning of the final estimator in the stacking model

refined_param_grid = {
    'final_estimator__max_depth': [10, 15],
    'final_estimator__min_samples_split': [5, 10]
}

# Perform grid search with cross-validation to find the best hyperparameters for the final estimator
refined_grid_search = GridSearchCV(test_model_2, refined_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Fit the grid search on the training data
refined_grid_search.fit(X_train_scaled, y_train)

# Get the best estimator from the grid search
test_model_3 = refined_grid_search.best_estimator_

# Fit the best estimator on the training data
test_model_3.fit(X_train_scaled, y_train)

# Predict the target values on the test data using the best estimator
y_pred_final = test_model_3.predict(X_test_scaled)

# Evaluate the best estimator model using various metrics
print(f"Test Model 3 Mean Absolute Error: {mean_absolute_error(y_test, y_pred_final)}")
print(f"Test Model 3 Mean Squared Error: {mean_squared_error(y_test, y_pred_final)}")
print(f"Test Model 3 Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_final))}")
print(f"Test Model 3 R2 Score: {r2_score(y_test, y_pred_final)}")

Test Model 3 Mean Absolute Error: 1.026217640274395
Test Model 3 Mean Squared Error: 2.208554775067059
Test Model 3 Root Mean Squared Error: 1.4861207134910202
Test Model 3 R2 Score: 0.9554399280992413


In [38]:
# Initialize and fit XGBoost Regressor with specified parameters

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=6, learning_rate=0.1)

# Fit the model
xgb_model.fit(X_train_scaled, y_train)

# Predict the target values on the test data using XGBoost
y_pred_xgb = xgb_model.predict(X_test_scaled)

# Evaluate the XGBoost model using various metrics
print(f"XGBoost Mean Absolute Error: {mean_absolute_error(y_test, y_pred_xgb)}")
print(f"XGBoost Mean Squared Error: {mean_squared_error(y_test, y_pred_xgb)}")
print(f"XGBoost Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_xgb))}")
print(f"XGBoost R2 Score: {r2_score(y_test, y_pred_xgb)}")

XGBoost Mean Absolute Error: 0.9717069049170286
XGBoost Mean Squared Error: 1.7173539631732122
XGBoost Root Mean Squared Error: 1.310478524499052
XGBoost R2 Score: 0.9653504558990493


In [39]:
# Initialize and fit AdaBoost Regressor with specified parameters

ada_model = AdaBoostRegressor(n_estimators=100, learning_rate=0.1)


ada_model.fit(X_train_scaled, y_train)

# Predict the target values on the test data using AdaBoost
y_pred_ada = ada_model.predict(X_test_scaled)

# Evaluate the AdaBoost model using various metrics
print(f"AdaBoost Mean Absolute Error: {mean_absolute_error(y_test, y_pred_ada)}")
print(f"AdaBoost Mean Squared Error: {mean_squared_error(y_test, y_pred_ada)}")
print(f"AdaBoost Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_ada))}")
print(f"AdaBoost R2 Score: {r2_score(y_test, y_pred_ada)}")

AdaBoost Mean Absolute Error: 1.7584201620514244
AdaBoost Mean Squared Error: 5.241682838592406
AdaBoost Root Mean Squared Error: 2.289472174670923
AdaBoost R2 Score: 0.8942431644415255


In [40]:
# Define base models for ensemble
models = [
    ('xgb', xgb_model),
    ('dt', DecisionTreeRegressor()),
    ('tmodel1', test_model_1),
    ('tmodel3', test_model_3),
    ('adaboost',ada_model)
]

# Initialize and fit Voting Regressor as ensemble model with base models
ensemble_model = VotingRegressor(estimators=models)

# Fit
ensemble_model.fit(X_train_scaled, y_train)

# Predict the target values on the test data using the ensemble model
y_pred_ensemble = ensemble_model.predict(X_test_scaled)

# Evaluate the ensemble model using various metrics
print(f"Ensemble Model Mean Absolute Error: {mean_absolute_error(y_test, y_pred_ensemble)}")
print(f"Ensemble Model Mean Squared Error: {mean_squared_error(y_test, y_pred_ensemble)}")
print(f"Ensemble Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_ensemble))}")
print(f"Ensemble Model R2 Score: {r2_score(y_test, y_pred_ensemble)}")

Ensemble Model Mean Absolute Error: 0.987994477255898
Ensemble Model Mean Squared Error: 1.8828206788580102
Ensemble Model Root Mean Squared Error: 1.3721591302972154
Ensemble Model R2 Score: 0.9620119791579085


In [41]:
# Define parameter grid for Decision Tree Regressor

param_grid_dt = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform grid search with cross-validation to find the best hyperparameters for Decision Tree Regressor
grid_search_dt = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param_grid_dt, cv=5)
grid_search_dt.fit(X_train_scaled, y_train)

# Get the best estimator from the grid search for Decision Tree Regressor
best_dt = grid_search_dt.best_estimator_

# Define parameter grid for AdaBoost Regressor
param_grid_ada = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}

# Perform grid search with cross-validation to find the best hyperparameters for AdaBoost Regressor
grid_search_ada = GridSearchCV(estimator=AdaBoostRegressor(), param_grid=param_grid_ada, cv=5)
grid_search_ada.fit(X_train_scaled, y_train)

# Get the best estimator from the grid search for AdaBoost Regressor
best_ada = grid_search_ada.best_estimator_

# Update the models list with the best estimators
models = [
    ('xgb', xgb_model),
    ('dt', best_dt),
    ('tmodel1', test_model_1),
    ('tmodel3', test_model_3),
    ('adaboost', best_ada)
]

# Ensemble model with weights
weights = [1, 1, 1, 1, 1]
ensemble_model = VotingRegressor(estimators=models, weights=weights)

# Fit the ensemble model on the training data and predict on the test data
ensemble_model.fit(X_train_scaled, y_train)
y_pred_ensemble = ensemble_model.predict(X_test_scaled)

# Evaluate the ensemble model using various metrics
print(f"Ensemble Model Mean Absolute Error: {mean_absolute_error(y_test, y_pred_ensemble)}")
print(f"Ensemble Model Mean Squared Error: {mean_squared_error(y_test, y_pred_ensemble)}")
print(f"Ensemble Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_ensemble))}")
print(f"Ensemble Model R2 Score: {r2_score(y_test, y_pred_ensemble)}")

Ensemble Model Mean Absolute Error: 1.0255791744129588
Ensemble Model Mean Squared Error: 1.9478272323528942
Ensemble Model Root Mean Squared Error: 1.3956458119282609
Ensemble Model R2 Score: 0.960700398965081


In [42]:
# Define the parameter grid for Randomized Search on XGBoost model

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Perform randomized search with cross-validation for hyperparameter tuning of XGBoost model
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_grid, n_iter=50, cv=5, scoring='neg_mean_absolute_error',
                                   n_jobs=-1, random_state=42)
random_search.fit(X_train_scaled, y_train)

# Get the best model from the randomized search
best_model = random_search.best_estimator_

# Predict the target values on the test data using the best model
y_pred_best = best_model.predict(X_test_scaled)

# Evaluate the best model using various metrics
print(f"Tuned Ensemble Model Mean Absolute Error: {mean_absolute_error(y_test, y_pred_best)}")
print(f"Tuned Ensemble Model Mean Squared Error: {mean_squared_error(y_test, y_pred_best)}")
print(f"Tuned Ensemble Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_best))}")
print(f"Tuned Ensemble Model R2 Score: {r2_score(y_test, y_pred_best)}")

Tuned Ensemble Model Mean Absolute Error: 0.8187834846540729
Tuned Ensemble Model Mean Squared Error: 1.2602960650084272
Tuned Ensemble Model Root Mean Squared Error: 1.1226290861225836
Tuned Ensemble Model R2 Score: 0.9745721120856903


In [43]:
# Update the models list with the best models

models = [
    ('xgb', xgb_model),
    ('dt', DecisionTreeRegressor()),
    ('adaboost',ada_model),
    ('bm', best_model),
    ('bm2', ensemble_model)
]

# Initialize and fit a new Voting Regressor ensemble model with the updated models list
ensemble_model = VotingRegressor(estimators=models)
ensemble_model.fit(X_train_scaled, y_train)

# Predict the target values on the test data using the new ensemble model
y_pred_ensemble = ensemble_model.predict(X_test_scaled)

# Evaluate the new ensemble model using various metrics
print(f"Ensemble Model Mean Absolute Error: {mean_absolute_error(y_test, y_pred_ensemble)}")
print(f"Ensemble Model Mean Squared Error: {mean_squared_error(y_test, y_pred_ensemble)}")
print(f"Ensemble Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_ensemble))}")
print(f"Ensemble Model R2 Score: {r2_score(y_test, y_pred_ensemble)}")

Ensemble Model Mean Absolute Error: 0.9643374454751895
Ensemble Model Mean Squared Error: 1.7193105060556733
Ensemble Model Root Mean Squared Error: 1.3112248114094216
Ensemble Model R2 Score: 0.9653109804499893


In [45]:
players_22 = male_players_legacy[male_players_legacy_relevant_columns.index]

In [46]:
# Split the testing data into new training and testing sets for another evaluation

y1 = players_22['overall']
x1 = players_22.drop('overall', axis=1)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(x1, y1, test_size=0.2, random_state=42)

# Scale the new training and testing sets
X_train_new_scaled = scaler.fit_transform(X_train_new)
X_test_new_scaled = scaler.transform(X_test_new)


# Predict the target values on the new test data using the ensemble model
tester = ensemble_model.predict(X_test_new_scaled)

# Evaluate the ensemble model on the new test data using various metrics
print(f"Tuned Ensemble Model Mean Absolute Error: {mean_absolute_error(y_test_new, tester)}")
print(f"Tuned Ensemble Model Mean Squared Error: {mean_squared_error(y_test_new, tester)}")
print(f"Tuned Ensemble Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test_new, tester))}")
print(f"Tuned Ensemble Model R2 Score: {r2_score(y_test_new, tester)}")

C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(
C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\mcnob\Desktop\Ashesi A\Intro To AI\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was 

Tuned Ensemble Model Mean Absolute Error: 0.9643374454751895
Tuned Ensemble Model Mean Squared Error: 1.7193105060556733
Tuned Ensemble Model Root Mean Squared Error: 1.3112248114094216
Tuned Ensemble Model R2 Score: 0.9653109804499893


In [47]:
# Evaluate the ensemble model on the new test data using various metrics
joblib.dump(ensemble_model, 'stacking_model.pkl', compress=3)

['stacking_model.pkl']

In [48]:
players_22

,overall,movement_reactions,potential,passing,wage_eur,mentality_composure,value_eur,dribbling,attacking_short_passing,mentality_vision,international_reputation,skill_long_passing,ldm,rdm,cdm,power_shot_power
0,93.0,94.0,95.0,86.0,550000.0,59.0,100500000.0,96.0,89.0,90.0,5.0,76.0,65,65,65,80.0
1,92.0,90.0,92.0,81.0,375000.0,59.0,79000000.0,91.0,82.0,81.0,5.0,72.0,66,66,66,94.0
2,90.0,89.0,90.0,83.0,275000.0,59.0,54500000.0,92.0,86.0,84.0,5.0,76.0,67,67,67,86.0
3,90.0,85.0,90.0,81.0,275000.0,59.0,52500000.0,86.0,84.0,83.0,5.0,76.0,68,68,68,93.0
4,90.0,89.0,90.0,58.0,300000.0,59.0,63500000.0,63.0,42.0,20.0,5.0,41.0,43,43,43,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,39.0,61.0,40.0,700.0,40.0,110000.0,47.0,38.0,42.0,1.0,33.0,34,34,34,64.0
161579,46.0,42.0,58.0,29.0,750.0,35.0,110000.0,34.0,30.0,31.0,1.0,25.0,43,43,43,33.0
161580,46.0,50.0,58.0,43.0,500.0,35.0,110000.0,46.0,51.0,40.0,1.0,43.0,47,47,47,51.0
161581,46.0,45.0,70.0,36.0,500.0,43.0,150000.0,46.0,40.0,44.0,1.0,26.0,32,32,32,42.0
